在根目录新建concept文件夹，将解压的学业水平评估文件夹（原文件罗外学业评估数据.zip）放进去

在根目录新建detail文件夹，将解压的学业考试的明细文件夹（原文件2023-2024每周学业评估.zip）放进去

### 合并一周内各科目的知识点清单

每一周的的知识点清单由各科老师负责填写，被分成了若干的excel文档放在了各科目的文件夹中，需要将他们合并成一个excel文件，便于后续匹配题目知识点

In [13]:
# 基础版本
import pandas as pd
import os
import glob

# 基础路径和相关参数
grade = '二'  # 年级，例如 1, 2, 3
subjects = ['地理', '历史', '生物', '数学', '英语', '语文', '道法']  # 科目列表
base_path = r'D:\Common\罗外KT数据\test\concept'

# 用于拼接所有Excel文件的空数据框
combined_df = pd.DataFrame()

# 遍历每个周次，从1到17周
for week in range(2, 18):
    base_folder = fr'{base_path}\初{grade}年级学业水平评估(第{week}周)'  # 动态设置基础文件夹路径
    combined_df = combined_df[0:0]  # 清空数据框，准备存储新的数据
    
    # 遍历每个科目文件夹
    for subject in subjects:
        # 构建科目文件夹的路径
        folder_path = os.path.join(base_folder, subject)
        
        # 使用glob进行模糊匹配，找到文件夹中的xlsx文件
        file_pattern = os.path.join(folder_path, '*.xlsx')
        matched_files = glob.glob(file_pattern)
        
        if matched_files:
            print(matched_files)
            for file_path in matched_files:
                # 读取文件并从第二行开始，跳过第一行
                df = pd.read_excel(file_path, skiprows=0)
                # 读取文件并将内容添加到combined_df中
                combined_df = pd.concat([combined_df, df], ignore_index=True)
        else:
            print(f'文件夹 {folder_path} 中没有找到符合条件的文件，跳过该文件夹。')

    # 检查done文件夹是否存在，如果不存在则创建它
    done_folder = os.path.join(base_path, 'done')
    if not os.path.exists(done_folder):
        os.makedirs(done_folder)

    # 保存合并后的数据到新的Excel文件
    output_path = fr'{done_folder}\初{grade}年级学业水平评估知识点清单（第{week}周）.xlsx'
    combined_df.to_excel(output_path, index=False)


文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\地理 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\历史 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\生物 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\数学 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\英语 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\语文 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第2周)\道法 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\地理 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\历史 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\生物 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\数学 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\英语 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级学业水平评估(第3周)\语文 中没有找到符合条件的文件，跳过该文件夹。
文件夹 D:\Common\罗外KT数据\test\concept\初二年级

### 将每周的答题明细表数据进行格式转换

现有的得分明细数据是每个学生一行记录，包含所有题目的得分。现在需要转换成每个学生都有多行记录，每一题对应一行记录。

现在：

 学号 姓名  题1  题2  ...  题66 

 1   张三   1   0    ...   0   

目的：

 学号 姓名  题目  得分

 1   张三   题1  1   
 
 1   张三   题2  0   

...

1   张三   题66  0   



In [14]:
import pandas as pd
import os
import re

# Define the base folder path containing the subfolders with Excel files
base_folder_path = r'D:\Common\罗外KT数据\test\detail'

# Iterate over all subfolders in the base folder
for subfolder_name in os.listdir(base_folder_path):
    subfolder_path = os.path.join(base_folder_path, subfolder_name)
    
    # Only process if the current item is a directory
    if os.path.isdir(subfolder_path):
        
        # Extract grade and week from the subfolder name
        # 单科成绩-科学-初二15周学业评估-海云天测试学校2023
        if('第' in subfolder_name):
            match = re.match(r'单科成绩-科学-初([一二三])第(\d+)周学业评估', subfolder_name)
            if match:
                grade_str = match.group(1)
                week_str = match.group(2)

                # Convert grade from Chinese character to number
                chinese_numerals = {'一': 1, '二': 2, '三': 3}
                grade_num = chinese_numerals.get(grade_str, 0)

                # Convert week to two-digit string
                week_num = f'{int(week_str):02d}'

                # Create the time string (grade-week)
                time_str = f'{grade_num}{week_num}'
        else:
            match = re.match(r'单科成绩-科学-初([一二三])(\d+)周学业评估', subfolder_name)
            if match:
                grade_str = match.group(1)
                week_str = match.group(2)

                # Convert grade from Chinese character to number
                chinese_numerals = {'一': 1, '二': 2, '三': 3}
                grade_num = chinese_numerals.get(grade_str, 0)

                # Convert week to two-digit string
                week_num = f'{int(week_str):02d}'

                # Create the time string (grade-week)
                time_str = f'{grade_num}{week_num}'
        

        # Iterate over all files in the subfolder
        for file_name in os.listdir(subfolder_path):
            if file_name.endswith('.xlsx') and '小题分' in file_name:
                # Construct full file path
                file_path = os.path.join(subfolder_path, file_name)

                # 提取学校名字 (e.g., "海云天测试学校2023")
                school_name = file_name.split('-')[-1].split('.')[0]

                # Load the Excel file
                df = pd.read_excel(file_path)

                # Step 1: Select columns we need
                id_columns = ['考号', '姓名', '学校名称', '班级名称']
                objective_columns = [col for col in df.columns if col.startswith('客观_')]

                # Step 2: Melt the dataframe to long format for objective scores
                df_long = df.melt(id_vars=id_columns, value_vars=objective_columns, 
                                  var_name='题号', value_name='得分')

                # Step 3: Rename columns to match desired output
                df_long.rename(columns={
                    '考号': '考号',
                    '姓名': '姓名',
                    '学校名称': '学校名称',
                    '班级名称': '班级名称',
                    '题号': '题号',
                    '得分': '得分'
                }, inplace=True)

                # Step 4: Replace '客观_1', '客观_2', etc., with just numbers
                df_long['题号'] = df_long['题号'].str.replace('客观_', '').astype(int)

                # Step 5: Add the new column "时间（年级-周）"
                df_long['时间（年级-周）'] = time_str

                # Create the 'done' folder if it doesn't exist
                done_folder = os.path.join(base_folder_path, 'done')
                if not os.path.exists(done_folder):
                    os.makedirs(done_folder)

                # Construct the new file name by concatenating grade, week, school name, and "已转换明细"
                new_file_name = f"初{grade_str}年级第{week_str}周{school_name}已转换明细.xlsx"
                output_file_path = os.path.join(done_folder, new_file_name)

                # Save the transformed dataframe to a new Excel file
                df_long.to_excel(output_file_path, index=False)


e:\environment\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
e:\environment\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
e:\environment\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
e:\environment\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
e:\environment\Python\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default sty

### 将知识点清单的知识点根据题号匹配到答题明细表中，并将匹配好的内容都拼接在一个文件中

上述操作分别得到每周的题目知识点对应关系，以及每周的答题明细，现在需要将知识点清单的知识点根据题号匹配到答题明细表中，并将匹配好的内容都拼接在一个文件中

In [16]:
import os
import pandas as pd
import re

# Define the paths for the folders
concept_folder = r"D:\Common\罗外KT数据\test\concept\done"
detail_folder = r"D:\Common\罗外KT数据\test\detail\done"
output_folder = r"D:\Common\罗外KT数据\test\match"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of all files in the two folders
concept_files = [f for f in os.listdir(concept_folder) if f.endswith('.xlsx')]
detail_files = [f for f in os.listdir(detail_folder) if f.endswith('.xlsx')]

# Initialize an empty DataFrame to store all merged results
all_merged_df = pd.DataFrame()

# Iterate through detail files to find matching concept files
for detail_file in detail_files:
    # Extract grade and week information from the detail file name using regex
    match = re.search(r'(\u521d\u4e00).*?(\u7b2c\d+\u5468)', detail_file)
    if match:
        grade, week = match.groups()
        
        # Find the corresponding concept file based on grade and week
        for concept_file in concept_files:
            if grade in concept_file and week in concept_file:
                # Load the matching detail and concept files
                detail_file_path = os.path.join(detail_folder, detail_file)
                concept_file_path = os.path.join(concept_folder, concept_file)
                
                details_df = pd.read_excel(detail_file_path)
                knowledge_df = pd.read_excel(concept_file_path)
                
                # Cleaning up the knowledge dataframe to keep only necessary columns
                knowledge_df_clean = knowledge_df[['题号', '知识点']]
                
                # Merging the two dataframes based on the '题号' column
                merged_df = details_df.merge(knowledge_df_clean, on='题号', how='left')
                
                # Append the merged dataframe to the cumulative DataFrame
                all_merged_df = pd.concat([all_merged_df, merged_df], ignore_index=True)
                
                # Print a confirmation message
                print(f"Merged file processed: {detail_file} with {concept_file}")
                break  # Move to the next detail file once a match is found

# Save the final concatenated DataFrame to the output folder
output_file_path = os.path.join(output_folder, "all_merged_results.xlsx")
all_merged_df.to_excel(output_file_path, index=False)

print("All matching and merging completed. Final merged file saved.")


All matching and merging completed. Final merged file saved.
